# Preprocessing

##### Package Imports

In [1]:
import time
import threading
from datetime import date, datetime, timedelta

##### Chart Null values

In [2]:
def find_null(df):
    count = df.isnull().sum().values
    total = df.shape[0]
    ratio = count/total
    return pd.DataFrame(df={'missing_count':count, 'missing_ratio':ratio}, index=df.columns.values)

##### UTC to Date Function

In [3]:
def dateconvert(df):
    
    df['Date'] = ''
    for i in range (0,len(df)):
        df['Date'][i] = time.strftime('%Y-%m-%d', time.localtime(df['created_utc'][i]))
        
    df.drop_duplicates(subset ="created_utc", inplace = True)

##### Missing Dates Function

In [4]:
def missingdates(df, year):
    #https://stackoverflow.com/questions/10688006/generate-a-list-of-datetimes-between-an-interval
    #Credit to Martjin Peters

    def perdelta(start, end, delta):
        curr = start
        while curr < end:
            yield curr
            curr += delta
    
    date_list = []
    for result in perdelta(date(year, 1, 1), date((year+1), 1, 1), timedelta(days=1)):
        date_list.append(str(result))
            
    date_df = df.groupby('Date').agg('sum')
    date_df['date'] = date_df.index
    
    for i in range(len(date_df)):
        for j in range(len(date_list)-1):
            if date_df['date'][i] == date_list[j]:
                date_list.remove(date_list[j])
                
    return date_list

##### Merge Texts Function

In [5]:
def mergetexts(df):
    
    for i in range(0, len(df['title'])):
        if pd.isnull(df['selftext'][i]) == True:
            pass
        elif bit['selftext'][i] == '[removed]':
            pass
        else:
            df['title'][i] = df['title'][i] + df['selftext'][i]